In [87]:
# COMP30027 Machine learning Project 1: Gaining Information about Naive Bayes
# Author: Jordan Ung <jordanu@student.unimelb.edu.au> [729938]
# Last Modified: 31.03.19

#### GENERAL GUIDE
# Read In And Inspect The Data
# Check for missing value - (1) delete rows with missing values, (2) Impute the missing values with dataset
# Check for anomalously extreme values

# QUESTIONS TO ASK:
# Should I handle datasets with Missing values and ones without differently?
# In tht case do I need to hardcode it or iterate through the whole dataset once
# to find presence of missing values?

In [100]:
# Preprocess takes the name of a file and returns a list of instances
# within that file, with each instance containing a list of attributes
def preprocess(file_name):
    dataset = []
    with open(file_name, 'r') as file:
        # Add each instance to a list to be used later
        for line in file.readlines():
            dataset.append(line.strip().split(','))
            
    # Group all the instances with the same class together
    dataset = sorted(dataset, key=lambda x: x[-1])
    return dataset

data_file = 'anneal.csv'
missing_value_files = ['breast-cancer.csv', 'hepatitis.csv', 'hypothyroid.csv', 'mushroom.csv', 'primary-tumor.csv']

In [167]:
# Train takes a list of instances and returns a 3-tuple, containing
# the classes, a list of counts of each class in the dataset and a
# list of each class with a list inside each of every attribute,
# containing a dictionary of the count of each unique attribute value
def train(instance_list, missing_value):
    data_info = ([], [], [])
#     classes = {}
    
#     for instance in instance_list:
#         # Check for missing values and delete rows containing them
#         if missing_value in instance:
#             instance_list.remove(instance)
            
#         class_type = instance[-1]
#         # Calculate the frequencies of all the class values
#         if class_type not in classes:
#             classes[class_type] = 1
#             data_info[0].append(class_type)
#         else:
#             classes[class_type] += 1
#     print(classes)
            
#     # Create a data structure to store all the attribute probabilities w.r.t each class
#     for i in range(len(classes)):
#         data_info[2].append([])
#         for j in range(len(instance_list[0]) - 1):
#             data_info[2][i].append({})

#     # Amount of instances that are of a certain class
#     for class_value in classes:
#         data_info[1].append(classes[class_value])
        
        
#     print(data_info)
#     for i in data_info[2]:
#         print(len(i))
    
    current_class = instance_list[0][-1]
    data_info[2].append({})
    class_index = 0
    number_of_instances = 0
    # Tally each value in each attribute for each class
    for data in instance_list:
        attribute_num = 0

        # New class has been detected
        if data[-1] != current_class:
            # Add data structure to support new class
            data_info[2].append({})
            class_index += 1
            current_class = data[-1]
            
            # Input information from previous class to data structure
            data_info[0].append(current_class)
            data_info[1].append(number_of_instances)
            number_of_instances = 0
            
            # Add attribute for each 
        
        # Input each instance's attribute into the appropriate dictionary
        for attribute in data[:-1]:
#             print("Class Index:", class_index, "|", data_info[0][class_index])
#             print("Attribtue Value", attribute)
#             print("Attribute Number:", attribute_num)
#             print("--------------------")
            if attribute in data_info[2][class_index][attribute_num]:
                data_info[2][class_index][attribute_num][attribute] += 1
            else:
                data_info[2][class_index][attribute_num][attribute] = 1
            attribute_num += 1
        
        number_of_instances += 1
        

    return data_info # Return the 3-tuple
# train(preprocess(missing_value_files[0]), "?")

In [168]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(model, instances):
    predicted_classes = []
    
    # Find class prediction for each class
    for data in instances:
        attribute_number = 0
        probability_of_class = []
        
        # Find probability of an instance being of a particular class
        for class_index in range(len(model[2])):
            sum = 1.0
            for attribute in model[2][class_index]:
                if data[attribute_number] in attribute:
                    sum *= attribute[data[attribute_number]] / model[1][class_index]
                else:
                    sum /= len(model[2][class_index])
            probability_of_class.append(sum)
        
        class_index = 0
        highest_probability = 0
        # Find the class with the highest probability
        for i in range(len(probability_of_class)):
            if probability_of_class[i] > highest_probability:
                highest_probability = probability_of_class[i]
                class_index = i
#                 print("Higher")
#             elif probability_of_class[i] < highest_probability:
#                 print("Less Than")
#             else:
#                 print("Equal Too")
        predicted_classes.append(model[0][class_index])
    return predicted_classes # Return a list of predicted classes

# predict(train(preprocess(missing_value_files[1]), "?"), preprocess(missing_value_files[1]))

In [169]:
# Evaluates certain metrics of performance of the predictor model
def evaluate(predictions, dataset):
    tries = 0
    correct = 0
    
    for i in range(len(predictions)):
        if predictions[i] == dataset[i][-1]:
            correct += 1
#         else:
#             print("My attempt:", predictions[i], "whereas the correct class was", dataset[i][-1])
        tries += 1
    print("-----------------------------------")
    print("Correct:", correct, "out of", tries, "attempts.")
    print("Accuracy Rate (%): ", correct / tries * 100)
    print("-----------------------------------")
    return
# preprocess(missing_value_files[-1])
train(preprocess(missing_value_files[-1]), "?")

# for i in missing_value_files:
#     evaluate(predict(train(preprocess(i), "?"), preprocess(i)), preprocess(i))
# evaluate(predict(train(preprocess(missing_value_files[-1]), "?"), preprocess(missing_value_files[-1])), preprocess(missing_value_files[-1]))

# evaluate(predict(train(preprocess(data_file), "?"), preprocess(data_file)), preprocess(data_file))



KeyError: 0

In [166]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain():
    
    return

In [14]:
a = [{}] * 5

b = []
for i in range(5):
    b.append({})
    

b[1]['key'] = 'value'
a[1]['key'] = 'value'
print(a)
print(b)

[{'key': 'value'}, {'key': 'value'}, {'key': 'value'}, {'key': 'value'}, {'key': 'value'}]
[{}, {'key': 'value'}, {}, {}, {}]
